In [1]:
import os

import h5py
import numpy as np
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import pandas as pd
import seaborn as sns
sns.set_context('notebook')

import sklearn
from sklearn import datasets
from sklearn.metrics.pairwise import pairwise_kernels
import graphtools
#from pydiffmap.diffusion_map import DiffusionMap
import phate

os.chdir('..')
from diffusion_maps import compute_dm, plot_change_in_diffusion_distance, compute_diffusion_distance_direct, compute_diffusion_distance_using_dmcoords, display_powers_of_diff_op
from graph_utils import get_graph_representation, plot_graph, get_groups_from_graph
import manifold_methods
import plotting
import mappings
import data_loader

In [2]:
exp_path = '/lustre06/project/6065672/shared/DietNet/1KGB_POP24/1KGP/WGS30X_V1/oldnow/MattsPlace'
fname = '1000G.2504_WGS30x.GSA17k_MHI.intersectGSA.miss10perc.maf0.05.pruned.autosomes.noHLA.phased_imputed_V1.hdf5'

In [3]:
with h5py.File(os.path.join(exp_path, fname), 'r') as hf:
    #model_attrs = hf['gradients'][:]
    #print('loaded gradient of fc1 w.r.t. input from {}'.format(attr_fc1_saliency_name))
    inputs = hf['inputs'][:]
    class_label_names = hf['class_label_names'][:]
    class_labels = hf['class_labels'][:]
    samples = hf['samples'][:]
    snp_names = hf['snp_names'][:]
    
# make labels
label_with_names = [str(class_label_names[y])[2:-1] for y in class_labels]
class_label_names = [str(label)[2:-1] for label in class_label_names]

label_with_superpop_names = np.zeros_like(label_with_names)
for label in mappings.super_pops_1000G:
    index = pd.DataFrame(label_with_names).isin(mappings.super_pops_1000G[label]).values.flatten()
    label_with_superpop_names[index] = label

In [4]:
graphs = []
kernels = []

# How do our runs differ?
for i in range(5):
    G = graphtools.Graph(inputs, n_pca=100, n_landmark=None)
    K = G.kernel
    K = np.array(K.todense())
    P = G.diff_op
    graphs.append(G)
    kernels.append(K)
    #P_arr = np.array(P.todense())[new_order][:,new_order]

Look at Disconnections in each (directed) graph

In [5]:
for K in kernels:
    indices = get_groups_from_graph(K)
    print(np.unique(indices, return_counts=True)) # topologies change!

(array([0., 1., 2., 3.]), array([1428,  504,  381,    9]))
(array([0., 1., 2., 3.]), array([1428,  504,  381,    9]))
(array([0., 1., 2., 3.]), array([1428,  504,  381,    9]))
(array([0., 1., 2., 3.]), array([1428,  504,  381,    9]))
(array([0., 1., 2., 3.]), array([1809,  401,    9,  103]))
